# Задание 3

In [5]:
import re
import nltk
import json
import string
import pymorphy2
from string import Template
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\пк\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
indexes = {}
morph = pymorphy2.MorphAnalyzer(lang='ru')
for i in range(0, 100):
    words = []
    with open('pages/page№' + str(i + 1) + '.txt', encoding="utf-8") as file:
        page_strip = [row.strip().lower() for row in file]
    words_line = []
    for page_row in page_strip:
        page_row = [token for token in page_row.split() if token not in stopwords.words('russian') and token != " " and token.strip() not in string.punctuation]
        word_symbols = []
        for word in page_row:
            word_symbols.append(word)
        words_line += [word_symbols]   
    for word in words_line:
        words += word
    words = [w for w in filter(re.compile("^[а-яА-Я]+$").match, words)]
    words = list(dict.fromkeys(words))
    for index in words:
        if index in indexes.keys():
            indexes[index].append(i + 1)
        else:
            indexes[index] = [i + 1]

with open('inverted_index.txt', "w", encoding="utf-8") as inverted_index_file:
    for index_elem in indexes:
        t = Template('{\"count\":$count,\"inverted_array\":[$array],\"word\":\"$word\"}\n')
        inverted_index_file.write(t.substitute(count = str(len(indexes[index_elem])),
                                          array = ','.join([str(x) for x in indexes[index_elem]]),
                                          word = index_elem))

In [8]:
words_set = []
searching_text = input().lower()
searching_words = [token for token in searching_text.split() if token not in stopwords.words('russian') and token != " " and token.strip() not in string.punctuation]
word_symbols = []
words_line = []
for word in searching_words:
    word_symbols.append(word)
words_line += [word_symbols]
for word in words_line:
    words_set += word
words_set = [w for w in filter(re.compile("^[а-яА-Я]+$").match, words_set)]
t = Template('Поиск по словам : $words\n')
print(t.substitute(words = ','.join([str(x) for x in words_set])))

with open('inverted_index.txt', encoding="utf-8") as file:
    rows = [row.strip() for row in file]
count_words = {}
for row in rows:
    jsonData = json.loads(row.replace('\'', '\"'))
    count_words[jsonData["word"]] = jsonData["inverted_array"]

mas_res = {}
for word in words_set:
    mas_res[word] = count_words[word] if word in count_words.keys() else []

counter = 0
result = []
for elem in mas_res:
    counter += 1
    result = mas_res[elem] if counter == 1 else list(set(result) & set(mas_res[elem]))

with open('search_result.txt', "w", encoding="utf-8") as f:
    for key in result:
        try:
            f.write(str(key) + '\n')
        except Exception as e:
            print( "Something went wrong...\nerror: %s" % e)
    f.close()
    
t = Template('Найдено в файлах с номерами : $pages\n')
print(t.substitute(pages = ','.join([str(x) for x in result])))
print(result)

индустрия среднего
Поиск по словам : индустрия,среднего

Найдено в файлах с номерами : 1,3,25

[1, 3, 25]
